In [1]:
!ls /kaggle/input/00-setup-01-baseline-and-importance
!ls /kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning
!ls /kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning/artifacts
!ls /kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning/results


custom.css	    __output__.json   semantic-llm-pruning
__notebook__.ipynb  __results__.html
artifacts  models  results
head_importance.npy  head_ranking.json	mlp_importance.npy  mlp_ranking.json
ablation_studies  latency_baseline.json  perplexity_baseline.json


Charger les rankings

In [2]:
BASE_IN  = "/kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning"
BASE_OUT = "/kaggle/working/semantic-llm-pruning"

In [3]:
import json, numpy as np

with open(f"{BASE_IN}/artifacts/head_ranking.json") as f:
    head_rank = json.load(f)

mlp_importance = np.load(f"{BASE_IN}/artifacts/mlp_importance.npy")


0) Paths (IN/OUT) + dossiers

In [4]:
import os, json
import numpy as np
import torch

BASE_IN  = "/kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning"
BASE_OUT = "/kaggle/working/semantic-llm-pruning"

os.makedirs(f"{BASE_OUT}/models", exist_ok=True)
os.makedirs(f"{BASE_OUT}/results/ablation_studies", exist_ok=True)

ART_IN = f"{BASE_IN}/artifacts"
print("IN artifacts:", ART_IN)


IN artifacts: /kaggle/input/00-setup-01-baseline-and-importance/semantic-llm-pruning/artifacts


1) Charger le ranking des heads (Notebook 01 output)

In [5]:
with open(f"{ART_IN}/head_ranking.json", "r") as f:
    head_rank = json.load(f)

num_layers = head_rank["num_layers"]
num_heads  = head_rank["num_heads"]
ranking    = head_rank["ranking"]

print("Layers:", num_layers, "Heads/layer:", num_heads, "Ranking entries:", len(ranking))


Layers: 22 Heads/layer: 32 Ranking entries: 704


2) Construire la liste des heads à garder (top 80% par layer)

3) Recharger le modèle de base (obligatoire)

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_ID = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    dtype=torch.float16,
).to("cuda")

model.eval()


print("Model device:", next(model.parameters()).device)


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

2026-01-07 11:13:17.133920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767784397.590984      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767784397.697394      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767784398.727723      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767784398.727766      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767784398.727769      55 computation_placer.cc:177] computation placer alr

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Model device: cuda:0


In [7]:
bad = []
for name, p in model.named_parameters():
    if p.device.type != "cuda":
        bad.append((name, str(p.device)))
print("Non-cuda params:", bad[:5], "count:", len(bad))


Non-cuda params: [] count: 0


✅ Étape 4 (refaite proprement) — Head pruning -20% Q+O seulement, compatible GQA + même device/dtype

4.0 Préparer old_q_heads et head_dim

In [8]:
num_layers = model.config.num_hidden_layers

old_q_heads = model.config.num_attention_heads
kv_heads = getattr(model.config, "num_key_value_heads", None)

head_dim = getattr(model.config, "head_dim", None)
if head_dim is None:
    attn0 = model.model.layers[0].self_attn
    head_dim = attn0.q_proj.out_features // old_q_heads

print("old_q_heads:", old_q_heads, "| kv_heads:", kv_heads, "| head_dim:", head_dim)

old_q_heads: 32 | kv_heads: 4 | head_dim: 64


In [9]:
PRUNE_RATIO = 0.20

target = int(old_q_heads * (1.0 - PRUNE_RATIO))

if kv_heads is not None and kv_heads > 0:
    target = target - (target % kv_heads)
    target = max(target, kv_heads)
else:
    target = max(target, 1)

KEEP_PER_LAYER = target
print("KEEP_PER_LAYER (valid):", KEEP_PER_LAYER)


KEEP_PER_LAYER (valid): 24


In [10]:
# --- build per-layer q_scores from `ranking` (list of dicts: layer/head/score) ---
old_q_heads = model.config.num_attention_heads
old_kv_heads = getattr(model.config, "num_key_value_heads", None)
assert old_kv_heads is not None, "Ce patch suppose un modèle GQA (num_key_value_heads existe)."

# objectif: tu veux garder KEEP_PER_LAYER têtes Q par couche (comme avant)
NEW_Q_HEADS = KEEP_PER_LAYER

# propose un pruning KV proportionnel, puis ajuste pour que NEW_Q_HEADS % NEW_KV_HEADS == 0
NEW_KV_HEADS = max(1, int(round(old_kv_heads * (NEW_Q_HEADS / old_q_heads))))
while NEW_KV_HEADS > 1 and (NEW_Q_HEADS % NEW_KV_HEADS != 0):
    NEW_KV_HEADS -= 1

NEW_N_REP = NEW_Q_HEADS // NEW_KV_HEADS
print("Plan:", "old_q=", old_q_heads, "old_kv=", old_kv_heads, "| new_q=", NEW_Q_HEADS, "new_kv=", NEW_KV_HEADS, "new_n_rep=", NEW_N_REP)

# q_scores[l][h] = score
q_scores = [[0.0 for _ in range(old_q_heads)] for _ in range(num_layers)]
for item in ranking:
    l = item["layer"]
    h = item["head"]
    s = float(item.get("score", item.get("importance", 0.0)))
    q_scores[l][h] = s

group_size = old_q_heads // old_kv_heads

kv_keep_heads = {}
q_keep_heads = {}

for l in range(num_layers):
    # 1) score KV = moyenne des scores Q du groupe KV
    kv_scores = []
    for g in range(old_kv_heads):
        start = g * group_size
        end = (g + 1) * group_size
        kv_scores.append((g, sum(q_scores[l][start:end]) / max(1, (end - start))))

    kv_scores.sort(key=lambda x: x[1], reverse=True)
    kv_keep = sorted([g for g, _ in kv_scores[:NEW_KV_HEADS]])
    kv_keep_heads[l] = kv_keep

    # 2) pour chaque KV gardé, garder NEW_N_REP têtes Q DANS ce groupe
    q_keep = []
    for g in kv_keep:
        start = g * group_size
        end = (g + 1) * group_size
        heads = list(range(start, end))
        heads.sort(key=lambda h: q_scores[l][h], reverse=True)
        q_keep.extend(heads[:NEW_N_REP])

    q_keep = sorted(q_keep)
    q_keep_heads[l] = q_keep

print("Example layer0 kv_keep:", kv_keep_heads[0])
print("Example layer0 q_keep count:", len(q_keep_heads[0]), "heads:", q_keep_heads[0])


Plan: old_q= 32 old_kv= 4 | new_q= 24 new_kv= 3 new_n_rep= 8
Example layer0 kv_keep: [0, 1, 2]
Example layer0 q_keep count: 24 heads: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


4.1 Helpers “like” (même device/dtype)

4.2 Fonction pruning Q+O (GQA-safe)

In [11]:
import torch
import torch.nn as nn

def _prune_linear_out(linear: nn.Linear, out_idx: torch.Tensor) -> nn.Linear:
    W = linear.weight.data.index_select(0, out_idx).contiguous()
    b = linear.bias.data.index_select(0, out_idx).contiguous() if linear.bias is not None else None
    new = nn.Linear(linear.in_features, out_idx.numel(), bias=(b is not None)).to(W.device, W.dtype)
    new.weight.data.copy_(W)
    if b is not None:
        new.bias.data.copy_(b)
    return new

def _prune_linear_in(linear: nn.Linear, in_idx: torch.Tensor) -> nn.Linear:
    W = linear.weight.data.index_select(1, in_idx).contiguous()
    b = linear.bias.data.contiguous() if linear.bias is not None else None
    new = nn.Linear(in_idx.numel(), linear.out_features, bias=(b is not None)).to(W.device, W.dtype)
    new.weight.data.copy_(W)
    if b is not None:
        new.bias.data.copy_(b)
    return new

def _head_to_dim_indices(head_ids, head_dim: int, device):
    idx = []
    for h in head_ids:
        base = h * head_dim
        idx.extend(range(base, base + head_dim))
    return torch.tensor(idx, device=device, dtype=torch.long)

@torch.no_grad()
def prune_gqa_qkvo(attn, q_keep_heads, kv_keep_heads, head_dim: int):
    device = attn.q_proj.weight.device
    q_out_idx  = _head_to_dim_indices(q_keep_heads, head_dim, device)
    kv_out_idx = _head_to_dim_indices(kv_keep_heads, head_dim, device)

    # prune OUT
    attn.q_proj = _prune_linear_out(attn.q_proj, q_out_idx)
    attn.k_proj = _prune_linear_out(attn.k_proj, kv_out_idx)
    attn.v_proj = _prune_linear_out(attn.v_proj, kv_out_idx)

    # prune IN
    attn.o_proj = _prune_linear_in(attn.o_proj, q_out_idx)

    # patch attributs éventuels
    for attr in ["num_heads", "num_attention_heads"]:
        if hasattr(attn, attr):
            setattr(attn, attr, len(q_keep_heads))
    if hasattr(attn, "head_dim"):
        attn.head_dim = head_dim

    return len(q_keep_heads), len(kv_keep_heads)


4.3 Appliquer à tous les layers

In [12]:
new_q_list = []
new_kv_list = []

for l in range(num_layers):
    attn = model.model.layers[l].self_attn
    new_q, new_kv = prune_gqa_qkvo(attn, q_keep_heads[l], kv_keep_heads[l], head_dim)
    new_q_list.append(new_q)
    new_kv_list.append(new_kv)

print("New q-heads (layer0):", new_q_list[0])
print("New kv-heads (layer0):", new_kv_list[0])
print("All layers same q?", len(set(new_q_list)) == 1)
print("All layers same kv?", len(set(new_kv_list)) == 1)


New q-heads (layer0): 24
New kv-heads (layer0): 3
All layers same q? True
All layers same kv? True


In [13]:
attn = model.model.layers[0].self_attn
print("q_heads =", model.config.num_attention_heads)
print("kv_heads =", getattr(model.config, "num_key_value_heads", None))
print("q_proj.out =", attn.q_proj.out_features)
print("k_proj.out =", attn.k_proj.out_features)
print("head_dim =", attn.q_proj.out_features // model.config.num_attention_heads)
print("n_rep =", model.config.num_attention_heads // getattr(model.config, "num_key_value_heads", 1))
print("divisible?", model.config.num_attention_heads % getattr(model.config, "num_key_value_heads", 1) == 0)


q_heads = 32
kv_heads = 4
q_proj.out = 1536
k_proj.out = 192
head_dim = 48
n_rep = 8
divisible? True


In [14]:
attn0 = model.model.layers[0].self_attn
print("q_out:", attn0.q_proj.out_features)
print("k_out:", attn0.k_proj.out_features)
print("v_out:", attn0.v_proj.out_features)
print("o_in :", attn0.o_proj.in_features)
print("devices:",
      attn0.q_proj.weight.device,
      attn0.k_proj.weight.device,
      attn0.o_proj.weight.device)


q_out: 1536
k_out: 192
v_out: 192
o_in : 1536
devices: cuda:0 cuda:0 cuda:0


5) Mettre à jour la config (important pour save/load)

In [15]:
model.config.num_attention_heads = NEW_Q_HEADS
setattr(model.config, "num_key_value_heads", NEW_KV_HEADS)
setattr(model.config, "head_dim", head_dim)

print("Config:",
      "q_heads =", model.config.num_attention_heads,
      "kv_heads =", getattr(model.config, "num_key_value_heads", None),
      "head_dim =", model.config.head_dim,
      "n_rep =", model.config.num_attention_heads // getattr(model.config, "num_key_value_heads", 1),
      "divisible =", (model.config.num_attention_heads % getattr(model.config, "num_key_value_heads", 1) == 0))


Config: q_heads = 24 kv_heads = 3 head_dim = 64 n_rep = 8 divisible = True


6) Sanity check rapide (forward + génération)

In [16]:
@torch.no_grad()
def quick_generate(model, tokenizer, prompt="Explain attention pruning.", max_new_tokens=40):
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(quick_generate(model, tokenizer))


Explain attention pruning.

    # Explain why we use attention.
    # Explain why we use attention.
    # Explain why we use attention.
    # Explain why we use attention


7) Sauvegarder le modèle pruné (Notebook 02 output)

In [17]:
import os

save_dir = f"{BASE_OUT}/models/pruned_heads_20"
os.makedirs(save_dir, exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved pruned model to:", save_dir)

Saved pruned model to: /kaggle/working/semantic-llm-pruning/models/pruned_heads_20


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tmp_tok = AutoTokenizer.from_pretrained(save_dir, use_fast=True)
tmp_model = AutoModelForCausalLM.from_pretrained(save_dir, dtype=torch.float16).to("cuda").eval()

print(quick_generate(tmp_model, tmp_tok, prompt="Give 2 bullet points about pruning.", max_new_tokens=40))


Give 2 bullet points about pruning.
I'm not sure if this is a good idea or not.
I'm not sure if this is a good idea.
I'm not sure if this is a good idea


In [19]:
import torch, math
from datasets import load_dataset

@torch.no_grad()
def compute_ppl_wikitext2_zip_style(
    model,
    tokenizer,
    split="test",
    seq_len=1024,
    stride=512,
    max_windows=256,
    num_samples=None,
    max_length=None,
):
    if max_length is not None:
        seq_len = int(max_length)
    if num_samples is not None:
        max_windows = int(num_samples)

    model.eval()
    ds = load_dataset("wikitext", "wikitext-2-raw-v1", split=split)
    text = "\n\n".join(ds["text"])
    enc = tokenizer(text, return_tensors="pt")
    input_ids = enc["input_ids"][0]

    device = next(model.parameters()).device
    input_ids = input_ids.to(device)

    nll_sum = 0.0
    tok_count = 0
    nb = 0

    for start in range(0, input_ids.numel() - 1, stride):
        end = min(start + seq_len, input_ids.numel())
        x = input_ids[start:end].unsqueeze(0)

        labels = x.clone()
        if start > 0:
            overlap = min(stride, labels.size(1))
            labels[:, :-overlap] = -100

        out = model(input_ids=x, labels=labels, use_cache=False, return_dict=True)
        n_eval = (labels != -100).sum().item()

        nll_sum += out.loss.detach().float().item() * max(n_eval, 1)
        tok_count += n_eval

        nb += 1
        if nb >= max_windows or end == input_ids.numel():
            break

    ppl = float(torch.exp(torch.tensor(nll_sum / max(tok_count, 1))).item())
    return ppl


In [20]:
import time

@torch.no_grad()
def measure_latency(model, tokenizer, prompt="Explain attention pruning.", gen_tokens=64, runs=3):
    device = next(model.parameters()).device
    inp = tokenizer(prompt, return_tensors="pt").to(device)

    # warmup
    _ = model.generate(**inp, max_new_tokens=gen_tokens)

    times = []
    for _ in range(runs):
        torch.cuda.synchronize()
        t0 = time.time()
        _ = model.generate(**inp, max_new_tokens=gen_tokens)
        torch.cuda.synchronize()
        t1 = time.time()
        times.append((t1 - t0) * 1000 / gen_tokens)

    return sum(times) / len(times)


In [21]:
def count_params(model):
    return sum(p.numel() for p in model.parameters())


In [22]:
import pandas as pd
import os

ppl = compute_ppl_wikitext2_zip_style(model, tokenizer, num_samples=200, max_length=512)
lat_ms = measure_latency(model, tokenizer)
params = count_params(model)

print("PPL:", ppl)
print("Latency (ms/token):", lat_ms)
print("Params:", params)


README.md: 0.00B [00:00, ?B/s]

wikitext-2-raw-v1/test-00000-of-00001.pa(…):   0%|          | 0.00/733k [00:00<?, ?B/s]

wikitext-2-raw-v1/train-00000-of-00001.p(…):   0%|          | 0.00/6.36M [00:00<?, ?B/s]

wikitext-2-raw-v1/validation-00000-of-00(…):   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

PPL: 17.405532836914062
Latency (ms/token): 27.193083117405575
Params: 1048143872


In [23]:
OUT_CSV = f"{BASE_OUT}/results/ablation_studies/head_pruning.csv"
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

row = {
    "model": "TinyLlama-1.1B",
    "pruning_type": "head_qo",
    "prune_ratio": 0.20,
    "num_attention_heads": model.config.num_attention_heads,
    "num_key_value_heads": getattr(model.config, "num_key_value_heads", None),
    "params": params,
    "perplexity_wikitext2": ppl,
    "latency_ms_per_token": lat_ms,
}

df_new = pd.DataFrame([row])

if os.path.exists(OUT_CSV):
    df_old = pd.read_csv(OUT_CSV)
    df = pd.concat([df_old, df_new], ignore_index=True)
else:
    df = df_new

df.to_csv(OUT_CSV, index=False)
print("Saved metrics to:", OUT_CSV)
df


Saved metrics to: /kaggle/working/semantic-llm-pruning/results/ablation_studies/head_pruning.csv


,model,pruning_type,prune_ratio,num_attention_heads,num_key_value_heads,params,perplexity_wikitext2,latency_ms_per_token
0,TinyLlama-1.1B,head_qo,0.2,24,3,1048143872,17.405533,27.193083


In [24]:
!cat /kaggle/working/semantic-llm-pruning/results/ablation_studies/head_pruning.csv


model,pruning_type,prune_ratio,num_attention_heads,num_key_value_heads,params,perplexity_wikitext2,latency_ms_per_token
TinyLlama-1.1B,head_qo,0.2,24,3,1048143872,17.405532836914062,27.193083117405575


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
